Basic word (token) counts for XML responses.

I will note, wherever necessary, assumptions related to the stopwords corpora. For this count, mimetypes, namespaces and URNs (specifically those related to EPSG codes and the like). Also ditching numbers and timestamps.

Given that we are looking at a simple token count, the initial BoW vectors will include some errant punctuation. Punctuation-only tokens are dropped but things like '(This' are left as is.


In [1]:
%reload_ext autoreload
%autoreload 2


import glob
import json
import re
import dateutil.parser as dateparser
from itertools import chain

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import WordListCorpusReader

from semproc.rawresponse import RawResponse
from semproc.bag_parser import BagParser

In [2]:
def convert_header_list(headers):
    '''
    convert from the list of strings, one string
    per kvp, to a dict with keys normalized
    '''
    return dict(
        (k.strip().lower(), v.strip()) for k, v in (
            h.split(':', 1) for h in headers)
    )


def remove_stopwords(text):
    '''
    remove any known english stopwords from a
    piece of text (bag of words or otherwise)
    '''
    _stopwords = set(stopwords.words('english'))
    words = word_tokenize(text)
    words = words if isinstance(words, list) else words.split()
    return ' '.join([w for w in words if w not in _stopwords and w])


def load_token_list(term_file):
    '''
    load some stopword list from the corpus
    '''
    __location__ = '../corpora/'
    tokens = WordListCorpusReader(__location__, term_file)
    return [w.replace('+', '\+') for w in tokens.words()]


def remove_tokens(term_file, text):
    '''
    do this before something like tokenize or the
    resplit option will split the mimetypes to not
    be recognizable as such anymore
    '''
    words = load_token_list(term_file)

    pttn = re.compile('|'.join(words))
    return pttn.sub('', text)

def remove_numeric(text):
    match_pttn = ur'\w*\b-?\d\s*\w*'
    captures = re.findall(match_pttn, u' {0} '.format(text))

    # strip them out
    if captures:
        text = re.sub('|'.join(captures), ' ', text)
        return '' if text == '0' else text

    return text

def strip_dates(text):
        # this should still make it an invalid date
        # text = text[3:] if text.startswith('NaN') else text
        try:
            d = dateparser.parse(text)
            return ''
        except ValueError:
            return text
        except OverflowError:
            return text
        
def strip_filenames(text):
    # we'll see
    exts = ('png', 'jpg', 'hdf', 'xml', 'doc', 'pdf', 'txt', 'jar', 'nc', 'XSL', 'kml', 'xsd')
    return '' if text.endswith(exts) else text
    
def strip_identifiers(texts):
    # chuck any urns, urls, uuids
    _pattern_set = [
        ('url', ur"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""),
        # a urn that isn't a url
        ('urn', ur"(?![http://])(?![https://])(?![ftp://])(([a-z0-9.\S][a-z0-9-.\S]{0,}\S:{1,2}\S)+[a-z0-9()+,\-.=@;$_!*'%/?#]+)"),
        ('uuid', ur'([a-f\d]{8}(-[a-f\d]{4}){3}-[a-f\d]{12}?)'),
        ('doi', ur"(10[.][0-9]{4,}(?:[/][0-9]+)*/(?:(?![\"&\\'])\S)+)"),
        ('md5', ur"([a-f0-9]{32})")
    ]
    for pattern_type, pattern in _pattern_set:
        for m in re.findall(re.compile(pattern), texts):
            m = max(m) if isinstance(m, tuple) else m
            try:
                texts = texts.replace(m, '')
            except Exception as ex:
                print ex
                print m
                
    files = ['cat_interop_urns.txt', 'mimetypes.txt', 'namespaces.txt']
    for f in files:
        texts = remove_tokens(f, texts)
    return texts.split()

def remove_punctuation(text):
    simple_pattern = r'[;|>+:=.,()/?!\[\]{}]'
    text = re.sub(simple_pattern, ' ', text)
    text = text.replace(' - ', ' ').strip()
    return text if text != '-' else ''

def strip_punctuation(text):
    terminal_punctuation = '(){}[].,~|":'
    return text.strip(terminal_punctuation).strip()
    
def clean(text):
    text = strip_dates(text)
    text = remove_numeric(text)
    
    text = remove_punctuation(text.strip()).strip()
    text = strip_punctuation(text)
    
    text = strip_filenames(text)
    
    return text
        
exclude_tags = ['schemaLocation', 'noNamespaceSchemaLocation']

In [ ]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [ ]:
# the sql template for any xml document
# any identification or aggregation happens later
sketchy_sql = '''
select r.id, r.source_url, r.source_url_sha, r.cleaned_content
from responses r
where r.format = 'xml'
limit %s
offset %s;
'''


In [4]:
files = glob.glob('/Users/sparky/Documents/solr_responses/solr_20150922_docs/2b*.json')

for f in files[10:20]:
    with open(f, 'r') as g:
        data = json.loads(g.read())
        
    print data.get('url')
    
    headers = convert_header_list(data.get('response_headers', []))
    content_type = headers.get('content-type', '')
    
    rr = RawResponse(data.get('raw_content'), content_type)
    content = rr.clean_raw_content() # .decode('string_escape')
    
    # strip the html cruft but ignore the a tags
    bp = BagParser(content, True, False)
    if not bp.parser.xml:
        print 'NOT XML: ', content[:100]
        continue
    # we don't care about the fully qualified namespace here
    stripped_text = [b[1].split() for b in bp.strip_text(exclude_tags) if b[1]]
    stripped_text = list(chain.from_iterable(stripped_text))
    cleaned_text = [s for s in stripped_text if clean(s)]
    
    bow = strip_identifiers(' '.join(cleaned_text))
    
    print
    print stripped_text
    print cleaned_text
    print bow
    

http://findingaids.princeton.edu/collections/C1296/c0115.rdf

['Prayer', 'for', 'Binding', 'the', 'Evil', 'Eye', '(Ayn', 't,', 'Ayn', 'T', 'ela,', 'T', 'rq', 'l', 'm,', 'Ayn', 'W', 'rq,', 'Ayn', 'Nas,', 'Ayn', 'Z', 'm', 'd', 'W', 'ba', 'ed', '("The', 'Evil', 'Eye', 'of', 'Family', 'and', 'of', 'Strangers"),', 'Meq', 'a', '("The', 'Envious")', 'and', 'T', 'nkol', 'a', '("The', 'Deceitful"),', 'G', 'rgari,', 'etc', '1830-01-01T00:00:00Z', u'http://www.w3.org/2001/XMLSchema#dateTime', '1870-12-31T23:59:59Z', u'http://www.w3.org/2001/XMLSchema#dateTime', u'http://findingaids.princeton.edu/repositories/mss', u'http://id.loc.gov/vocabulary/iso639-2/eng', u'http://findingaids.princeton.edu/collections/C1296/c0114']
['Prayer', 'for', 'Binding', 'the', 'Evil', 'Eye', '(Ayn', 'Ayn', 'ela,', 'rq', 'l', 'Ayn', 'W', 'rq,', 'Ayn', 'Nas,', 'Ayn', 'Z', 'd', 'W', 'ba', 'ed', '("The', 'Evil', 'Eye', 'Family', 'Strangers"),', 'Meq', 'a', '("The', 'Envious")', 'nkol', 'a', '("The', 'Deceitful"),', 'G', 'r

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:17: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
//anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.w3.org/2001/XMLSchema#dateTime" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
//anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://findingaids.princeton.edu/repositories/mss" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks l

[u'2', '0085288', '1.1', '2013-12-25', '01:29:01+00', 'mare', '/nodc/archive/arc0041/0085288/1.1/', '11', '120160422', 'NODC-Readme.txt', '1aa8d73d906b820f8f3460b304215b17', u'md5', '42d3e4677f04d8b256b110018e5541e1dd89eacf', u'sha1', '7b64d8de0cc632362642100dfcb02bf75e540ab1620ffb17e990dbbfc81b23be', u'sha256', '19cef9e1b00b7ce287bd9e1073cd54788faebd3152b65df8395df6fcafdd7b697dc20a2769f0234aa3d3207104010c9c', u'sha384', '9a4775c7161a3aefe10c92783bff4c5bf2c7aecaab66aea5b043408077645e9d4bccbf88ae3c8fd0d57fbae5883ff9cb0f5436b8bb2c20816326bc2fc2b80027', u'sha512', 'regular', 'text', '-rw-r--r--', '1', 'root', 'root', '2129', '32768', '8', '2009-01-29', '15:52:27+00', '2012-02-17', '19:50:08+00', '2013-12-25', '01:29:01+00', '2013-12-25', '01:29:01+00', 'about', 'd41d8cd98f00b204e9800998ecf8427e', u'md5', 'da39a3ee5e6b4b0d3255bfef95601890afd80709', u'sha1', 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855', u'sha256', '38b060a751ac96384cd9327eb1b1e36a21fdb71114be07434c0cc7

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://datastore.hri.fi/Helsinki/kartat/1937_noin_Helsingin_kartta.TIF" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
//anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://dev.hel.fi/geoserver-preview?layers=1937_opaskartta" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' 

[u'https://www.opendata.fi/data/en/dataset/helsingin-kaupungin-kartta-noin-vuodelta-1937', u'urn:uuid:7f5e302f-f159-4854-a7c5-f703c25ca054', 'Helsingin', 'kaupungin', 'kartta', 'noin', 'vuodelta', '1937.', 'Karttaan', 'on', 'merkitty', 'kaupunginosat', 'ja', 'korttelit,', 'julkiset', 'rakennukset,', 'rakennetut', 'ja', 'rakentamattomat', 'alueet', 'sek', 'puistot', 'ja', 'istutukset.', 'My', 's', 'rautatie', 'sek', 'kaupungin', '12', 'raitiotielinjaa', 'n', 'kyv', 't', 'kartassa.', 'Kartan', 'mittakaava', 'on', '1:8000', '(?)', 'ja', 'kuvan', 'resoluutio', '300', 'dpi.', 'historia', 'kartat', u'https://www.opendata.fi/data/en/dataset/helsingin-kaupungin-kartta-noin-vuodelta-1937', 'helsingin-kaupungin-kartta-noin-vuodelta-1937', 'helsingin-kaupungin-kartta-noin-vuodelta-1937', 'Helsingin', 'kaupungin', 'kartta', 'noin', 'vuodelta', '1937', u'http://datastore.hri.fi/Helsinki/kartat/1937_noin_Helsingin_kartta.TIF', 'TIFF', 'TIFF', u'http://dev.hel.fi/geoserver-preview?layers=1937_opaskar

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#CI_DateTypeCode" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
//anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#MD_ProgressCode" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed th